In [1]:
jiant_url = "https://drive.google.com/u/0/uc?id=1mYZMjHUtuTB2ueS3Tn69dZvKZ4nVRpKy&export=download"
!wget "$jiant_url" -O jiant.zip
!unzip jiant.zip -d ./
!rm jiant.zip

%cd jiant
!pip install -r requirements-no-torch.txt
!pip install --no-deps -e ./

--2021-06-21 21:33:38--  https://drive.google.com/u/0/uc?id=1mYZMjHUtuTB2ueS3Tn69dZvKZ4nVRpKy&export=download
Resolving drive.google.com (drive.google.com)... 108.177.98.101, 108.177.98.138, 108.177.98.113, ...
Connecting to drive.google.com (drive.google.com)|108.177.98.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/h5dm3djpblb8r7gboi7v2754h0v2msnv/1624311150000/13675844253040106205/*/1mYZMjHUtuTB2ueS3Tn69dZvKZ4nVRpKy?e=download [following]
--2021-06-21 21:33:39--  https://doc-10-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/h5dm3djpblb8r7gboi7v2754h0v2msnv/1624311150000/13675844253040106205/*/1mYZMjHUtuTB2ueS3Tn69dZvKZ4nVRpKy?e=download
Resolving doc-10-18-docs.googleusercontent.com (doc-10-18-docs.googleusercontent.com)... 74.125.195.132, 2607:f8b0:400e:c09::84
Connecting to doc-10-18-docs.googleusercontent.com (do

Obtaining file:///content/jiant
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Running setup.py develop for jiant


Restart runtime after installing libs

In [1]:
%cd /content/jiant

import jiant.utils.python.io as py_io
import jiant.utils.display as display
import os

def init_task_config(task_names):
  for task_name in task_names:
    os.makedirs("/content/tasks/configs/", exist_ok=True)
    os.makedirs(f"/content/tasks/data/{task_name}", exist_ok=True)

    py_io.write_json({
      "task": task_name,
      "paths": {
        "train": f"/content/tasks/data/{task_name}/train.jsonl",
        "val":   f"/content/tasks/data/{task_name}/val.jsonl",
      },
      "name": task_name
    }, f"/content/tasks/configs/{task_name}_config.json")

/content/jiant


In [2]:
task_names = ["ner", "semeval", "coref",  # supported by jiant 
...           "srl"]                      # additional task for other badge     
#...           "sup-squad", "ques-squad"]  # todo: define tasks in jiant 

init_task_config(task_names)

In [5]:
# contains data of all probing tasks above
task_data_url = "https://drive.google.com/u/0/uc?id=1-nlA_dTsAqQ5oO0vii2lMsm34yYMmn_f&export=download"
!wget "$task_data_url" -O /content/task_data.zip
!unzip /content/task_data.zip -d /content/tasks/
!rm /content/task_data.zip

--2021-06-21 21:36:51--  https://drive.google.com/u/0/uc?id=1-nlA_dTsAqQ5oO0vii2lMsm34yYMmn_f&export=download
Resolving drive.google.com (drive.google.com)... 74.125.142.102, 74.125.142.139, 74.125.142.138, ...
Connecting to drive.google.com (drive.google.com)|74.125.142.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ge7nqsjb95a5lfq2foe9ti0idsputnre/1624311375000/13675844253040106205/*/1-nlA_dTsAqQ5oO0vii2lMsm34yYMmn_f?e=download [following]
--2021-06-21 21:36:51--  https://doc-04-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ge7nqsjb95a5lfq2foe9ti0idsputnre/1624311375000/13675844253040106205/*/1-nlA_dTsAqQ5oO0vii2lMsm34yYMmn_f?e=download
Resolving doc-04-18-docs.googleusercontent.com (doc-04-18-docs.googleusercontent.com)... 74.125.195.132, 2607:f8b0:400e:c09::84
Connecting to doc-04-18-docs.googleusercontent.com (do

In [3]:
import jiant.proj.main.export_model as export_model

models = ["bert-base-uncased", "csarron/bert-base-uncased-squad-v1"]

for model in models:
  export_model.export_model(
      hf_pretrained_model_name_or_path=model,
      output_base_path=f"/content/models/{model}",
  )

Some weights of BertForPreTraining were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at csarron/bert-base-uncased-squad-v1 were not used when initializing BertForPreTraining: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing BertForPreTraining from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForPreTraining from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForPreTraining were not initialized from the model checkpoint at cs

In [4]:
import jiant.shared.caching as caching
import jiant.proj.main.tokenize_and_cache as tokenize_and_cache

# Tokenize and cache each task
def tokenize(task_names):
  for task_name in task_names:
    tokenize_and_cache.main(tokenize_and_cache.RunConfiguration(
        task_config_path=f"/content/tasks/configs/{task_name}_config.json",
        hf_pretrained_model_name_or_path="csarron/bert-base-uncased-squad-v1",
        output_dir=f"/content/cache/{task_name}",
        phases=["train", "val"],
    ))

tokenize(task_names)

NerTask
  [train]: /content/tasks/data/ner/train.jsonl
  [val]: /content/tasks/data/ner/val.jsonl


ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectur

ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT

SemevalTask
  [train]: /content/tasks/data/semeval/train.jsonl
  [val]: /content/tasks/data/semeval/val.jsonl


ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT



ModelArchitectures.BERT
ModelArchitectures.BERT

CorefTask
  [train]: /content/tasks/data/coref/train.jsonl
  [val]: /content/tasks/data/coref/val.jsonl


ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectur

ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT

SrlTask
  [train]: /content/tasks/data/srl/train.jsonl
  [val]: /content/tasks/data/srl/val.jsonl


ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectur

ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT



In [5]:
row = caching.ChunkedFilesDataCache(f"/content/cache/{task_names[3]}/train").load_chunk(0)[0]["data_row"]
print(row.input_ids)
print(row.tokens)
print(row.spans)
print(row.tokens[row.spans[0][0]: row.spans[0][1]+1])
#print(row.tokens[row.spans[1][0]: row.spans[1][1]+1])

[  101  2006  2285  1023  1010  1996  1000 14004  1011 10556 11631  5332
  5575  1000  3864  2234  2000  2019  2203  1012   102     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
['[CLS]', 'on', 'december', '9', ',', 'the', '"', 'taipei', '-', 'ka', '##oh', '##si', '##ung', '"', 'elections', 'came', 'to', 'an', 'end', '.', '[SEP]']
[[15 15]
 [16 18]]
['came']


In [6]:
import jiant.proj.main.scripts.configurator as configurator

def create_jiant_task_config(task_name, epochs=5):
  jiant_run_config = configurator.SimpleAPIMultiTaskConfigurator(
      task_config_base_path="/content/tasks/configs",
      task_cache_base_path="/content/cache",
      train_task_name_list=[task_name],
      val_task_name_list=[task_name],
      train_batch_size=8,
      eval_batch_size=2,
      epochs=epochs,
      num_gpus=1,
  ).create_config()
  os.makedirs("/content/run_configs/", exist_ok=True)
  py_io.write_json(jiant_run_config, f"/content/run_configs/{task_name}_run_config.json")
  #display.show_json(jiant_run_config)

In [7]:
# You may restart runtime when you modify the jiant source code

import jiant.proj.main.runscript as main_runscript

def run_probing_task(task_name, model_name="bert-base-uncased", num_layers=1, eval_every_steps=1000):
  run_args = main_runscript.RunConfiguration(
      jiant_task_container_config_path=f"/content/run_configs/{task_name}_run_config.json",
      output_dir=f"/content/runs/{task_name}",
      hf_pretrained_model_name_or_path=model_name,
      model_path=f"/content/models/{model_name}/model/model.p",
      model_config_path=f"/content/models/{model_name}/model/config.json",
      learning_rate=1e-5,
      eval_every_steps=eval_every_steps,
      do_train=True,
      do_val=True,
      do_save=True,
      force_overwrite=True,
      num_hidden_layers=num_layers,
  )
  return main_runscript.run_loop(run_args)

In [21]:
task_names = ["ner", "semeval", "coref",  # supported by jiant 
...           "srl"]                      # additional task for other badge     
#...           "sup-squad", "ques-squad"]  # todo: define new tasks in jiant 
models = ["bert-base-uncased", "csarron/bert-base-uncased-squad-v1"]
num_layers = list(range(1, 5))      # from 1 to 12 layers

# Hyperparameters
epochs = 5              # used in paper
eval_every_steps = 1000 # used in paper

result = "model\ttask\tlayer\tacc\n"

for model in models:
  for task in task_names:
    for n_layers in num_layers:
      create_jiant_task_config(task, epochs=epochs, eval_every_steps=eval_every_steps)
      probing_output = run_probing_task(task, model, n_layers)
      result += model + "\t" + task + "\t" + str(n_layers) + "\t" + str(probing_output[task]["metrics"]["minor"]["acc"]) + "\n"

  jiant_task_container_config_path: /content/run_configs/ner_run_config.json
  output_dir: /content/runs/ner
  hf_pretrained_model_name_or_path: bert-base-uncased
  model_path: /content/models/bert-base-uncased/model/model.p
  model_config_path: /content/models/bert-base-uncased/model/config.json
  model_load_mode: from_transformers
  num_hidden_layers: 1
  do_train: True
  do_val: True
  do_save: True
  do_save_last: False
  do_save_best: False
  write_val_preds: False
  write_test_preds: False
  eval_every_steps: 1000
  save_every_steps: 0
  save_checkpoint_every_steps: 0
  no_improvements_for_n_evals: 0
  keep_checkpoint_when_done: False
  force_overwrite: True
  seed: -1
  learning_rate: 1e-05
  adam_epsilon: 1e-08
  max_grad_norm: 1.0
  optimizer_type: adam
  no_cuda: False
  fp16: False
  fp16_opt_level: O1
  local_rank: -1
  server_ip: 
  server_port: 
device: cpu n_gpu: 0, distributed training: False, 16-bits training: False
Using seed: 589780810
{
  "jiant_task_container_confi

/content/jiant/jiant/proj/main/modeling/model_setup.py:177: UserWarning: The following weights were not loaded: dict_keys(['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias'])
  "The following weights were not loaded: {}".format(remainder_weights_dict.keys())



Loading Best



{
  "aggregated": 0.4722222222222222,
  "ner": {
    "loss": 0.6235455274581909,
    "metrics": {
      "major": 0.4722222222222222,
      "minor": {
        "acc": 0.9444444444444444,
        "f1_micro": 0.0,
        "acc_and_f1_micro": 0.4722222222222222
      }
    }
  }
}
  jiant_task_container_config_path: /content/run_configs/ner_run_config.json
  output_dir: /content/runs/ner
  hf_pretrained_model_name_or_path: bert-base-uncased
  model_path: /content/models/bert-base-uncased/model/model.p
  model_config_path: /content/models/bert-base-uncased/model/config.json
  model_load_mode: from_transformers
  num_hidden_layers: 2
  do_train: True
  do_val: True
  do_save: True
  do_save_last: False
  do_save_best: False
  write_val_preds: False
  write_test_preds: False
  eval_every_steps: 1000
  save_every_steps: 0
  save_checkpoint_every_steps: 0
  no_improvements_for_n_evals: 0
  keep_checkpoint_when_done: False
  force_overwrite: True
  seed: -1
  learning_rate: 1e-05
  adam_epsilon:


Loading Best


...


{
  "aggregated": 0.4678030303030303,
  "srl": {
    "loss": 0.6579315662384033,
    "metrics": {
      "major": 0.4678030303030303,
      "minor": {
        "acc": 0.9356060606060606,
        "f1_micro": 0.0,
        "acc_and_f1_micro": 0.4678030303030303
      }
    }
  }
}
  jiant_task_container_config_path: /content/run_configs/srl_run_config.json
  output_dir: /content/runs/srl
  hf_pretrained_model_name_or_path: csarron/bert-base-uncased-squad-v1
  model_path: /content/models/csarron/bert-base-uncased-squad-v1/model/model.p
  model_config_path: /content/models/csarron/bert-base-uncased-squad-v1/model/config.json
  model_load_mode: from_transformers
  num_hidden_layers: 3
  do_train: True
  do_val: True
  do_save: True
  do_save_last: False
  do_save_best: False
  write_val_preds: False
  write_test_preds: False
  eval_every_steps: 1000
  save_every_steps: 0
  save_checkpoint_every_steps: 0
  no_improvements_for_n_evals: 0
  keep_checkpoint_when_done: False
  force_overwrite: Tru


Loading Best



{
  "aggregated": 0.4803030303030303,
  "srl": {
    "loss": 0.6943320333957672,
    "metrics": {
      "major": 0.4803030303030303,
      "minor": {
        "acc": 0.8939393939393939,
        "f1_micro": 0.06666666666666668,
        "acc_and_f1_micro": 0.4803030303030303
      }
    }
  }
}
  jiant_task_container_config_path: /content/run_configs/srl_run_config.json
  output_dir: /content/runs/srl
  hf_pretrained_model_name_or_path: csarron/bert-base-uncased-squad-v1
  model_path: /content/models/csarron/bert-base-uncased-squad-v1/model/model.p
  model_config_path: /content/models/csarron/bert-base-uncased-squad-v1/model/config.json
  model_load_mode: from_transformers
  num_hidden_layers: 4
  do_train: True
  do_val: True
  do_save: True
  do_save_last: False
  do_save_best: False
  write_val_preds: False
  write_test_preds: False
  eval_every_steps: 1000
  save_every_steps: 0
  save_checkpoint_every_steps: 0
  no_improvements_for_n_evals: 0
  keep_checkpoint_when_done: False
  forc


Loading Best



{
  "aggregated": 0.5454545454545454,
  "srl": {
    "loss": 0.6635209619998932,
    "metrics": {
      "major": 0.5454545454545454,
      "minor": {
        "acc": 0.9242424242424242,
        "f1_micro": 0.16666666666666669,
        "acc_and_f1_micro": 0.5454545454545454
      }
    }
  }
}


In [22]:
print(result)

model	task	layer	acc
bert-base-uncased	ner	1	0.9444444444444444
bert-base-uncased	ner	2	0.9074074074074074
bert-base-uncased	ner	3	0.9259259259259259
bert-base-uncased	ner	4	0.9259259259259259
bert-base-uncased	semeval	1	0.8157894736842105
bert-base-uncased	semeval	2	0.8421052631578947
bert-base-uncased	semeval	3	0.8421052631578947
bert-base-uncased	semeval	4	0.868421052631579
bert-base-uncased	coref	1	0.5476190476190477
bert-base-uncased	coref	2	0.7619047619047619
bert-base-uncased	coref	3	0.7380952380952381
bert-base-uncased	coref	4	0.7857142857142857
bert-base-uncased	srl	1	0.9204545454545454
bert-base-uncased	srl	2	0.9204545454545454
bert-base-uncased	srl	3	0.9393939393939394
bert-base-uncased	srl	4	0.9545454545454546
csarron/bert-base-uncased-squad-v1	ner	1	0.9444444444444444
csarron/bert-base-uncased-squad-v1	ner	2	0.9444444444444444
csarron/bert-base-uncased-squad-v1	ner	3	0.9259259259259259
csarron/bert-base-uncased-squad-v1	ner	4	0.9444444444444444
csarron/bert-base-uncased-sq